In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import time
%matplotlib inline

### Load the data
-------

In [ ]:
details = json.loads(open("details.json", "r").read())
X_data_path = details['coin'] + "_" + details['prediction'] + "_data.csv"
Y_data_path = details['coin'] + "_" + details['prediction'] + "_data.csv"

X_data_df = pd.read_csv(X_data_path)
Y_data_raw = pd.read_csv(Y_data_path)

In [ ]:
#get only columns: open,	high,	low,	close
X_data_df = X_data_df.iloc[:,2:12]
Y_data_raw = Y_data_raw.iloc[:,2:12]

#show data with each column is a different color
X_data_df.iloc[:,0].plot(subplots=True, figsize=(20,10))


In [ ]:
#to numpy 
X_data_raw = X_data_df.to_numpy()
Y_data_raw = Y_data_raw.to_numpy()


### Calculate the true labels
---------

In [ ]:
labels = details['label']

#open + close / 2
Y_data = (Y_data_raw[:,0] + Y_data_raw[:,3]) / 2

# Y_data[i] - Y_data[i-1]
Y_data_diff = np.diff(Y_data)

#calculate the percentage change
Y_data = Y_data_diff / Y_data[:-1]

#show the data
plt.plot(Y_data)
plt.show()


In [ ]:
#Using k-means to cluster the data into 5 groups, scatter plot
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(Y_data.reshape(-1,1))
plt.scatter(Y_data, np.zeros(Y_data.shape), c=kmeans.labels_, cmap='rainbow')
plt.show()

Y = kmeans.labels_

### Calculate the features
---------

__Data cols:__
1. open
2. high
3. low
4. close
5. volume
6. close_time
7. quote_volume
8. trades
9. taker_buy_volume
10. taker_buy_quote_volume


In [ ]:
PERIOD = 96
#Open + High + Low + Close + Volume
X = np.zeros((X_data_raw.shape[0] - PERIOD, 19))
#First 5
X[:,0:5] = X_data_raw[PERIOD:,0:5]

#normalize Volume
X[:,4] = X_data_raw[PERIOD:,4] / X_data_raw[PERIOD:,4].max()

#SMA in span of 1 day. (96 point of data)
sma = X_data_df.iloc[:,3].rolling(window=PERIOD).mean()[PERIOD:]
X[:,5] = sma

#EMA in span of 1 day. (96 point of data)
#EMA on high
ema = X_data_df.iloc[:,1].ewm(span=PERIOD).mean()[PERIOD:]
X[:,6] = ema

#RSI in span of 1 day. (96 point of data)
delta = X_data_df.iloc[:,3].diff()
A = delta.rolling(PERIOD).mean()

# Calculate average gain and average loss
AG = np.where(delta > 0, abs(A), 0 )
AL =  np.where(delta < 0, abs(A), 1 )

# Calculate RS and RSI
RS = AG / AL
RSI = 100 - (100 / (1 + RS))
X[:,7] = RSI[PERIOD:]

In [ ]:
#Boilinger Bands
#Calculate the rolling mean and standard deviation
rolling_mean = X_data_df.iloc[:,3].rolling(window=PERIOD).mean()[PERIOD:]
rolling_std = X_data_df.iloc[:,3].rolling(window=PERIOD).std()[PERIOD:]

#Calculate the upper and lower Bollinger Bands
upper_band = rolling_mean + (rolling_std * 2)
lower_band = rolling_mean - (rolling_std * 2)
X[:,8] = upper_band
X[:,9] = lower_band

#MACD
#Calculate the short term exponential moving average (EMA)
ShortEMA = X_data_df.iloc[:,3].ewm(span=12, adjust=False).mean()[PERIOD:]

#Calculate the long term exponential moving average (EMA)
LongEMA = X_data_df.iloc[:,3].ewm(span=26, adjust=False).mean()[PERIOD:]

#Calculate the MACD line
MACD = ShortEMA - LongEMA
X[:,10] = MACD

#OBV
#Calculate the On Balance Volume
OBV = np.zeros(X_data_raw.shape[0] - PERIOD)
OBV[0] = X_data_raw[PERIOD,4]
for i in range(1, X_data_raw.shape[0] - PERIOD):
    if X_data_raw[i + PERIOD,3] > X_data_raw[i + PERIOD - 1,3]:
        OBV[i] = OBV[i-1] + X_data_raw[i + PERIOD,4]
    elif X_data_raw[i + PERIOD,3] < X_data_raw[i + PERIOD - 1,3]:
        OBV[i] = OBV[i-1] - X_data_raw[i + PERIOD,4]
    else:
        OBV[i] = OBV[i-1]
X[:,11] = OBV

#normalize OBV
X[:,11] = X[:,11] / X[:,11].max()


#ADX
#Calculate the True Range
TR = np.zeros(X_data_raw.shape[0])
for i in range(1, X_data_raw.shape[0] ):
    TR[i] = max(X_data_raw[i ,1] - X_data_raw[i ,2], abs(X_data_raw[i ,1] - X_data_raw[i  - 1,3]), abs(X_data_raw[i ,2] - X_data_raw[i  - 1,3]))

#Calculate the Positive Directional Indicator and Negative Directional Indicator
PDI = np.zeros(X_data_raw.shape[0] )
NDI = np.zeros(X_data_raw.shape[0] )
for i in range(1, X_data_raw.shape[0] ):
    PDI[i] = 100 * (X_data_raw[i ,1] - X_data_raw[i  - 1,3]) / TR[i]
    NDI[i] = 100 * (X_data_raw[i  - 1,3] - X_data_raw[i ,2]) / TR[i]

#Calculate the Average Directional Movement Index
PDI = pd.DataFrame(PDI).ewm(span=14).mean()
NDI = pd.DataFrame(NDI).ewm(span=14).mean()
DX = 100 * abs(PDI - NDI) / (PDI + NDI)
ADX = pd.DataFrame(DX).ewm(span=14).mean()
X[:,12] = ADX[PERIOD:].to_numpy().reshape(X.shape[0])

#normalize ADX
X[:,12] = X[:,12] / X[:,12].max()

#ATR
#Use the above TR
ATR = pd.DataFrame(TR).ewm(span=14).mean()
X[:,13] = ATR[PERIOD:].to_numpy().reshape(X.shape[0])


In [ ]:
#Fibonacci Retracement
#Calculate the Fibonacci Retracement
Fibonacci = np.zeros((X_data_raw.shape[0] - PERIOD, 5))
Fibonacci[:,0] = X_data_raw[PERIOD:,3] - X_data_raw[PERIOD:,2]
Fibonacci[:,1] = X_data_raw[PERIOD:,1] - X_data_raw[PERIOD:,3]
Fibonacci[:,2] = X_data_raw[PERIOD:,1] - X_data_raw[PERIOD:,2]
Fibonacci[:,3] = X_data_raw[PERIOD:,3] - X_data_raw[PERIOD:,0]
Fibonacci[:,4] = X_data_raw[PERIOD:,1] - X_data_raw[PERIOD:,0]

#normalize Fibonacci
Fibonacci = Fibonacci / Fibonacci.max()

X[:,14:19] = Fibonacci


In [ ]:
Y = Y[PERIOD-1:]

l = X.shape[0]

#only use half of the data
X = X[:l//2]
Y = Y[:l//2]


Y_ = np.zeros((Y.shape[0],5))
for i in range(Y.shape[0]):
    if Y[i] == 0:
        Y_[i,0] = 1
    elif Y[i] == 1:
        Y_[i,1] = 1
    elif Y[i] == 2:
        Y_[i,2] = 1
    elif Y[i] == 3:
        Y_[i,3] = 1
    elif Y[i] == 4:
        Y_[i,4] = 1
Y = Y_
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
class KNearestNeighbor():
    def __init__(self):
        pass

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X, distance_function, predict_function, k=1):
        dists  = distance_function(self.X_train, X)
        labels = predict_function(self.y_train, dists, k=k)
        return labels
    
    @staticmethod
    def evaluate(y_test_pred, y_test, isprint=True):
        num_test = len(y_test.flatten())
        num_correct = np.sum(y_test_pred.flatten() == y_test.flatten())
        accuracy = float(num_correct) / num_test
        if isprint: print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
        return accuracy

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

In [ ]:
num_test = X_test.shape[0]
def compute_distances_no_loop(X_train, X_test):
    result_mtx = np.sqrt(np.sum(X_test**2, axis=1, keepdims=True) + np.sum(X_train**2, axis=1) - 2 * X_test.dot(X_train.T))
    
    return result_mtx
def predict_labels(y_train, dist, k=1):
    result_mtx = np.argsort(dists_no, axis=1)[:, :k]
    result = []
    for i in range(num_test):
        result.append(np.bincount(y_train[result_mtx[i]]).argmax())
        
    return np.array(result)

# Test your implementation
dists_no = compute_distances_no_loop(X_train, X_test)
print("Train:", X_train.shape, "Test:",X_test.shape, "Result:",dists_no.shape)

y_test_pred = predict_labels(y_train, dists_no, k=1)
classifier.evaluate(y_test_pred, y_test)